<a href="https://colab.research.google.com/github/habibsain/Index-Option/blob/main/Option_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lightgbm

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
#import nsepython as ns
from lightgbm import LGBMClassifier
from statsmodels.graphics.tsaplots import plot_acf
import datetime as dt
import requests
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
tickers = ["^NSEI","^NSEBANK", "NIFTY_FIN_SERVICE.NS", "^BSESN"]
for tk in tickers:
  df = yf.download(tk, start = "2016-01-01", end = "2024-07-29", interval="1d")
  d= yf.Ticker(tk)
  d.info['shortName'].replace(" ", "_")
  df.to_csv(d.info['shortName'].replace(" ", "_")+".csv")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [ ]:
data = pd.read_csv("NIFTY_FIN_SERVICE.csv")
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-01-01,7057.950195,7103.049805,7024.399902,7091.450195,7091.450195,254900
1,2016-01-04,7058.250000,7058.250000,6901.950195,6909.299805,6909.299805,428200
2,2016-01-05,6942.299805,6942.299805,6873.549805,6891.649902,6891.649902,454100
3,2016-01-06,6878.350098,6925.500000,6829.149902,6854.250000,6854.250000,491500
4,2016-01-07,6785.950195,6785.950195,6691.899902,6704.000000,6704.000000,612000


In [ ]:
#plot
#k[['Close', 'ema']].plot( figsize = (16,8))

In [ ]:
#indicators(period = 2d)
def get_sma(price, period = 20, column = 'sma'):
    return price.rolling(window = period).mean()

def get_ema(price, period = 20, column = 'ema'):
    return price.ewm(span = period, adjust = False).mean()
length =14
def get_rsi(price, period = 20, column = 'rsi'):
    delta = price.diff()
    up = delta.clip(lower = 0)
    down = -1 * delta.clip(upper = 0).abs()

    roll_up = up.ewm(span = period, adjust = False).mean()
    roll_down = down.ewm(span = period, adjust = False).mean()
    rs = roll_up / roll_down
    rsi = 100.0 - (100.0 / (1.0 + rs))

    rsi[:] = np.select([roll_down == 0, roll_up == 0, True], [100, 0, rsi])

    return rsi


In [ ]:
#Label
def get_label(price, base, column = 'label'):
    price_diff = price - base
    output = price_diff/abs(price_diff)
    return output

In [ ]:
k = data.copy()
#k['sma'] = get_sma(k['Close'])
k['ema'] = get_ema(k['Close'])
k['rsi'] = get_rsi(k['Close'])

#Shifted down the base by 100 points
k['base'] = k.shift(-1)['Open']-100
k['label'] = get_label(k.shift(-1)['Close'], k.shift(-1)['Open'])
k['datetime'] = pd.to_datetime(k['Date'])
k.set_index('datetime', inplace = True)
#k.drop(columns=['Date', 'Close', 'Open', 'High', 'Low'], inplace=True)
k.drop(columns=['Close', 'Date'], inplace=True)
k.tail()

,Open,High,Low,Adj Close,Volume,ema,rsi,base,label
datetime,,,,,,,,,
2024-07-22,23543.199219,23719.050781,23450.800781,23626.800781,1086100,23449.206128,375.300148,23610.300781,-1.0
2024-07-23,23710.300781,23725.000000,23111.050781,23304.099609,1636100,23435.386460,-459.326708,23124.099609,-1.0
2024-07-24,23224.099609,23388.349609,22969.199219,23165.099609,1553400,23409.644855,-223.109473,22821.599609,1.0
2024-07-25,22921.599609,23064.199219,22866.150391,23049.500000,1215700,23375.345345,-151.495600,22841.150391,1.0
2024-07-26,22941.150391,23358.949219,22934.000000,23316.699219,1398500,23369.760000,-1297.341577,NaN,NaN


In [ ]:
k.head()

,Open,High,Low,Adj Close,Volume,ema,rsi,base,label
datetime,,,,,,,,,
2016-01-01,7057.950195,7103.049805,7024.399902,7091.450195,254900,7091.450195,NaN,6958.250000,-1.0
2016-01-04,7058.250000,7058.250000,6901.950195,6909.299805,428200,7074.102539,0.0,6842.299805,-1.0
2016-01-05,6942.299805,6942.299805,6873.549805,6891.649902,454100,7056.726097,0.0,6778.350098,-1.0
2016-01-06,6878.350098,6925.500000,6829.149902,6854.250000,491500,7037.442660,0.0,6685.950195,-1.0
2016-01-07,6785.950195,6785.950195,6691.899902,6704.000000,612000,7005.686216,0.0,6634.850098,-1.0


In [ ]:
k.dropna(inplace = True, ignore_index=True)
#k.drop(columns = ['Date'], inplace = True)
Df = k.copy()
Df.head()

,Open,High,Low,Adj Close,Volume,ema,rsi,base,label
0,7058.250000,7058.250000,6901.950195,6909.299805,428200,7074.102539,0.000000,6842.299805,-1.0
1,6942.299805,6942.299805,6873.549805,6891.649902,454100,7056.726097,0.000000,6778.350098,-1.0
2,6878.350098,6925.500000,6829.149902,6854.250000,491500,7037.442660,0.000000,6685.950195,-1.0
3,6785.950195,6785.950195,6691.899902,6704.000000,612000,7005.686216,0.000000,6634.850098,-1.0
4,6734.850098,6758.399902,6706.549805,6723.049805,488800,6978.768462,-1.320894,6560.549805,1.0


In [ ]:
y = Df["label"]
Df = Df.drop(columns = "label")

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(Df,y)
model1 = RandomForestClassifier()
model2 = LogisticRegression()
model3 = SVC()
model4 = LGBMClassifier()
model = [model1, model2, model3, model4]
for item in model:
  item.fit(X_train, Y_train)
  prediction = item.predict(X_test)
  print(accuracy_score(Y_test, prediction))

0.5142857142857142
0.5695238095238095
0.5657142857142857
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 771, number of negative: 801
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000473 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2040
[LightGBM] [Info] Number of data points in the train set: 1572, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.490458 -> initscore=-0.038173
[LightGBM] [Info] Start training from score -0.038173
0.4990476190476191


In [ ]:
t = yf.download("NIFTY_FIN_SERVICE.NS", start = "2024-07-24", end = "2024-07-31", interval="1d")
t.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-07-24,23224.099609,23388.349609,22969.199219,23165.099609,23165.099609,1553400
2024-07-25,22921.599609,23064.199219,22866.150391,23049.500000,23049.500000,1215700
2024-07-26,22941.150391,23358.949219,22934.000000,23316.699219,23316.699219,1398500
2024-07-29,23371.000000,23668.550781,23222.150391,23315.300781,23315.300781,1776300
2024-07-30,23298.900391,23583.849609,23208.349609,23338.000000,23338.000000,1491600


In [ ]:
#New Data
x1 = t.iloc[1]
x2 = t.iloc[2]

s = t.copy()
#s['sma'] = get_sma(s['Close'])
s['ema'] = get_ema(s['Close'])
s['rsi'] = get_rsi(s['Close'])
s['base'] = s.shift(-1)['Open']-100
s['label'] = get_label(s.shift(-1)['Close'], s.shift(-1)['Open'])
#s['f_close'] = s.shift(-1)['Close']
#s['datetime'] = pd.to_datetime(s['Date'])
#s.set_index('datetime', inplace = True)
#s.drop(columns=['Date', 'Close', 'Open', 'High', 'Low'], inplace=True)
s.drop(columns=['Close'], inplace=True)
s.head()

,Open,High,Low,Adj Close,Volume,ema,rsi,base,label
Date,,,,,,,,,
2024-07-24,23224.099609,23388.349609,22969.199219,23165.099609,1553400,23165.099609,NaN,22821.599609,1.0
2024-07-25,22921.599609,23064.199219,22866.150391,23049.500000,1215700,23154.090123,0.000000,22841.150391,1.0
2024-07-26,22941.150391,23358.949219,22934.000000,23316.699219,1398500,23169.576703,-32.154051,23271.000000,-1.0
2024-07-29,23371.000000,23668.550781,23222.150391,23315.300781,1776300,23183.455187,-32.094356,23198.900391,1.0
2024-07-30,23298.900391,23583.849609,23208.349609,23338.000000,1491600,23198.173741,-36.645614,NaN,NaN


In [ ]:
s.dropna(inplace = True, ignore_index=True)
#s.drop(columns = ['Date'], inplace = True)
s.reset_index()
s.head()

,Open,High,Low,Adj Close,Volume,ema,rsi,base,label
0,22921.599609,23064.199219,22866.150391,23049.500000,1215700,23154.090123,0.000000,22841.150391,1.0
1,22941.150391,23358.949219,22934.000000,23316.699219,1398500,23169.576703,-32.154051,23271.000000,-1.0
2,23371.000000,23668.550781,23222.150391,23315.300781,1776300,23183.455187,-32.094356,23198.900391,1.0


In [ ]:
v = s["label"][1:]
v = s.drop(columns = "label")
w = v[1:]
w

,Open,High,Low,Adj Close,Volume,ema,rsi,base
1,22941.150391,23358.949219,22934.000000,23316.699219,1398500,23169.576703,-32.154051,23271.000000
2,23371.000000,23668.550781,23222.150391,23315.300781,1776300,23183.455187,-32.094356,23198.900391


In [ ]:
X_test.head()

,Open,High,Low,Adj Close,Volume,ema,rsi,base
1444,17661.250000,17762.650391,17282.849609,17629.949219,937100,18489.603587,-44.370629,17458.300781
687,10584.400391,10609.049805,10443.099609,10545.049805,812400,10654.041333,-605.723876,10457.000000
1093,10039.500000,10219.900391,9965.750000,10007.099609,3232200,9885.230121,849.509876,9877.000000
25,6250.850098,6306.299805,6234.049805,6278.100098,665700,6451.753351,-83.447165,6128.899902
2007,20867.949219,21069.849609,20647.849609,20897.500000,1618400,20708.048333,355.956519,20828.400391


In [ ]:
prediction = model4.predict(w)
prediction

array([-1., -1.])

In [ ]:
plot_acf(k['label'], lags=30)
plt.show()

In [ ]:
#Using Lags
N = k.copy()
N['label_1day_prev'] = N['label'].shift(1)
N['label_2day_prev'] = N['label'].shift(2)

N.dropna(inplace = True, ignore_index=True)
y_N = N["label"]
X_N = N.drop(columns = "label")
X_N.head()

,Open,High,Low,Adj Close,Volume,ema,rsi,base,label_1day_prev,label_2day_prev
0,6878.350098,6925.500000,6829.149902,6854.250000,491500,7037.442660,0.000000,6685.950195,-1.0,-1.0
1,6785.950195,6785.950195,6691.899902,6704.000000,612000,7005.686216,0.000000,6634.850098,-1.0,-1.0
2,6734.850098,6758.399902,6706.549805,6723.049805,488800,6978.768462,-1.320894,6560.549805,-1.0,-1.0
3,6660.549805,6722.450195,6599.649902,6664.750000,566100,6948.861942,-1.264401,6596.299805,1.0,-1.0
4,6696.299805,6696.299805,6563.899902,6579.700195,569200,6913.703681,-1.182832,6517.549805,-1.0,1.0


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_N,y_N)
model1 = RandomForestClassifier()
model2 = LogisticRegression()
model3 = SVC()
model4 = LGBMClassifier()
model = [model1, model2, model3, model4]
for item in model:
  item.fit(X_train, Y_train)
  prediction = item.predict(X_test)
  print(accuracy_score(Y_test, prediction))

0.5419847328244275
0.5248091603053435
0.5248091603053435
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 750, number of negative: 821
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000341 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2046
[LightGBM] [Info] Number of data points in the train set: 1571, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.477403 -> initscore=-0.090450
[LightGBM] [Info] Start training from score -0.090450
0.5229007633587787
